# Main CNN model for bat call classification

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow_addons.metrics import F1Score
from keras import backend as K 
import cv2
import time
from sklearn.model_selection import train_test_split
import itertools_len as itertools
from itertools_len import product
import gc
from tensorflow.keras.optimizers.legacy import Adam, SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.model_selection import KFold

/Users/seba/Documents/main_data_cloud/oth/s3/ml/prj/ml_project/venv/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
# load image data s and reshape 
data = pd.read_pickle('./data/images_df_numerical.pkl')

# convert to numpy array
X, y = data['data'], data['Species']
classes = y.unique()
image_size = X[0].size
samples = X.size
image_shape = (216,334,3) # height, width , channel
# reshape every row to the image, swap rgbs and scale to 0-1
X = np.array([
    cv2.cvtColor(row.reshape(image_shape), cv2.COLOR_BGR2RGB).astype('float32')/255. 
    for row in X])
y = np.array([row.astype('int32') for row in y])

In [3]:
kfold = KFold(n_splits=10, shuffle=True)

tf.keras.utils.set_random_seed(1)

# If using TensorFlow, this will make GPU ops as deterministic as possible,
# but it will affect the overall performance, so be mindful of that.
tf.config.experimental.enable_op_determinism()

In [4]:
number_of_classes = classes.size
early_stopping = EarlyStopping(monitor='val_accuracy', patience=30, min_delta=0.001, start_from_epoch=15, restore_best_weights=True)
epochs = 200
dropout_rate = 0.4

def kaggle_model(optimizer):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=image_shape))
    model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(number_of_classes, activation='softmax'))
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    return model

def create_optimizers(X_train) -> dict:
    s = 130 * len(X_train) // 32 # number of steps in 130 epochs (batch size = 32)
    exp_decay_sgd_adagrad = ExponentialDecay(0.01, s, 0.1)
    exp_adam = ExponentialDecay(0.1, s, 0.95, staircase=True)

    momentum = 0.99
    sgd_exp = SGD(exp_decay_sgd_adagrad, momentum=momentum)
    adam_exp = Adam(exp_adam)

    sgd = SGD(0.001, momentum=momentum)
    adam = Adam(0.001)

    return {"sgd_exp": sgd_exp, "adam_exp": adam_exp, "sgd": sgd, "adam": adam}

histories_with_params = list()

# Training and validating the model using KFold
for train_indezes, test_indezes in kfold.split(X, y):
    X_train, y_train = X[train_indezes], y[train_indezes]
    X_test, y_test = X[train_indezes], y[test_indezes]

    optimizers = create_optimizers(X_train)

    for optimizer_name, optimizer in optimizers.items():
        K.clear_session()
        model = kaggle_model(optimizer)
        history = model.fit(
            X_train,
            y_train,
            epochs=epochs,
            batch_size=32,
            workers=1, # workers are number of cores
            callbacks=early_stopping,
            validation_split=0.2,
            verbose=1)
        model.save("cnn_files/model.keras", overwrite=True)
        history_with_param = {"optimizer": optimizer_name, "history": history}
        histories_with_params.append(history_with_param)
        del model
        K.clear_session()
        tf.reset_default_graph() 
        gc.collect()

number_of_epochs = len(history.history["accuracy"])
for history_with_param in histories_with_params:
    model = load_model(f"cnn_files/model_{history_with_param['optimizer']}.keras")
    test_score = round(model.evaluate(X_test, y_test)[1], 2)*100
    del model
    gc.collect()

    plt.figure()
    plt.plot(history_with_param["history"].history["accuracy"], label="train_data accuracy")
    plt.plot(history_with_param["history"].history["val_accuracy"], label="val_data accuracy")
    plt.scatter(number_of_epochs, test_score/100, label="test_data accuracy", marker="x", c="g")
    plt.title(f"opt: {history_with_param['optimizer']} Test Score: {test_score}%")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend(loc="upper left")
    plt.savefig(f"./cnn_files/{history_with_param['optimizer']}.png",dpi=600)
    #plt.show()


2023-12-21 16:27:35.172355: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-12-21 16:27:35.172655: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-12-21 16:27:35.172660: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-12-21 16:27:35.173172: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-21 16:27:35.173858: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/200


2023-12-21 16:27:39.942266: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


84/84 [==============================] - 10s 103ms/step - loss: 2.1313 - accuracy: 0.4909 - val_loss: 2.0261 - val_accuracy: 0.5522
Epoch 2/200
84/84 [==============================] - 6s 71ms/step - loss: 2.3471 - accuracy: 0.5804 - val_loss: 3.0720 - val_accuracy: 0.4582
Epoch 3/200
84/84 [==============================] - 6s 69ms/step - loss: 2.1771 - accuracy: 0.5984 - val_loss: 2.6127 - val_accuracy: 0.2030
Epoch 4/200
84/84 [==============================] - 6s 67ms/step - loss: 1.4275 - accuracy: 0.6450 - val_loss: 1.1222 - val_accuracy: 0.5687
Epoch 5/200
84/84 [==============================] - 6s 68ms/step - loss: 1.3455 - accuracy: 0.6820 - val_loss: 0.8426 - val_accuracy: 0.6507
Epoch 6/200
84/84 [==============================] - 6s 67ms/step - loss: 1.1519 - accuracy: 0.7137 - val_loss: 0.7878 - val_accuracy: 0.7463
Epoch 7/200
84/84 [==============================] - 6s 68ms/step - loss: 0.9448 - accuracy: 0.7137 - val_loss: 1.0211 - val_accuracy: 0.6194
Epoch 8/200
84/8

KeyboardInterrupt: 